In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import scipy.stats as stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
import pickle
import joblib
from imblearn.under_sampling import RandomUnderSampler

In [2]:
# Load data in chunks to handle large file size
chunk_size = 500000  # Adjust chunk size for memory efficiency
chunks = []

# Reading the dataset in chunks
for chunk in pd.read_csv(r'C:/Users/User/Desktop/MDT 28/Assignments/Microsoft -Classifying Cybersecurity Incidents/GUIDE_Test.csv/GUIDE_Test.csv', chunksize=chunk_size, low_memory=False):
    # Optimize memory usage by downcasting data types
    for col in chunk.select_dtypes(include=['int64', 'float64']).columns:
        chunk[col] = pd.to_numeric(chunk[col], downcast='integer')
    chunks.append(chunk)

# Concatenate all chunks into a single DataFrame
df = pd.concat(chunks, ignore_index=True)
del chunks  # Free up memory

In [3]:
features = ['Id',
 'OrgId',
 'IncidentId',
 'AlertId',
 'DetectorId',
 'AlertTitle',
 'DeviceId',
 'Sha256',
 'IpAddress',
 'Url',
 'Category',
 'IncidentGrade',
 'EntityType',
 'EvidenceRole']

df = df[features]

In [4]:
df.shape

(4147992, 14)

In [5]:
df = df.drop_duplicates()

In [6]:
df.isnull().sum()

Id               0
OrgId            0
IncidentId       0
AlertId          0
DetectorId       0
AlertTitle       0
DeviceId         0
Sha256           0
IpAddress        0
Url              0
Category         0
IncidentGrade    0
EntityType       0
EvidenceRole     0
dtype: int64

In [7]:
cat_col = ['Category', 'IncidentGrade', 'EntityType', 'EvidenceRole']

In [8]:
# Load the encoders dictionary from the pickle file
with open('label_encoders.pkl', 'rb') as file:
    loaded_encoders = pickle.load(file)

# Apply the encoders to the test data
for column in cat_col:
    try:
        df[column] = loaded_encoders[column].transform(df[column])
    except:
        continue

print("Encoders have been applied to the test data.")

Encoders have been applied to the test data.


In [9]:
X = df.drop('IncidentGrade', axis = 1)
y = df['IncidentGrade']
del df

In [10]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(X)

In [11]:
model = joblib.load('best_model.joblib')

In [12]:
test_data_pred = model.predict(X)

accuracy = accuracy_score(y, test_data_pred)
recall = recall_score(y, test_data_pred, average='macro')  # Adjust for binary/multiclass
f1 = f1_score(y, test_data_pred, average='macro')          # Adjust for binary/multiclass
precision = precision_score(y, test_data_pred, average='macro')  # Adjust for binary/multiclass


print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Precision: {precision}")

Accuracy: 0.9257820344308735
Recall: 0.9203978879535909
F1 Score: 0.9180690192423975
Precision: 0.915933680899041
